In [8]:
import os
import pandas as pd
import plotly.express as px

from plots.simulation.GE3_plots import sankeyplot, emission_data_dict_to_df
from ggmodel_dev.models.landuse import BE2_scenario, GE3_scenario
from ggmodel_dev.models.water import EW_scenario
from ggmodel_dev.models.transport import VEHC_scenario
from ggmodel_dev.projection import run_projection



def get_data_dict_from_folder_parquet(folder_name):
    files = os.listdir(folder_name)
    data_dict = {file.split('.')[0]: pd.read_parquet(
        f'{folder_name}/{file}') for file in files}
    data_dict = {name: df[name] for name, df in data_dict.items()}
    return data_dict


def get_data_dict_from_folder(folder_name):
    files = os.listdir(folder_name)
    data_dict = {file.split('.')[0]: pd.read_csv(
        f'{folder_name}/{file}') for file in files}

    data_dict = {name: df.set_index([col for col in df.columns if col != name]).squeeze(
    ) for name, df in data_dict.items()}

    return data_dict

In [9]:
scenario_data_dictionnary = {
    'EW_models': get_data_dict_from_folder('data/sim/EW'),
    'BE2_model': get_data_dict_from_folder('data/sim/BE2'),
    'GE3_model': get_data_dict_from_folder_parquet('data/sim/GE3'),
    'VEHC_model': get_data_dict_from_folder('data/sim/VEHC'),
}


In [244]:
from ggmodel_dev.projection import *
from ggmodel_dev.models.landuse.GE3 import model_dictionnary

projection_dict = {
    'Pop': lambda x: x,
    'OEi': lambda x: x,
    'EF_EEi': lambda x: x,
    'TAi': lambda x: x,
    'IN_F': lambda x: x,
    'MYi': lambda x: x,
    'EF_ASi': lambda x: x,
    'EF_Ti': lambda x: x,
    'EF_CH4Ti': lambda x: x,
    'EF_Li': lambda x: x,
    'EF_F': lambda x: x,        
}


def run_scenario(data_dict,
                 MM_Ti=1/3,
                 MM_ASi=1/3,
                 MM_LPi=1/3,
                 TAi_pct_Asses=1,
                 TAi_pct_Cattle_dairy=1,
                 TAi_pct_Cattle_non_dairy=1,
                 TAi_pct_Chickens_broilers=1,
                 TAi_pct_Chickens_layers=1,
                 TAi_pct_Ducks=1,
                 TAi_pct_Goats=1,
                 TAi_pct_Horses=1,
                 TAi_pct_Mules=1,
                 TAi_pct_Sheep=1,
                 TAi_pct_Swine_breeding=1,
                 TAi_pct_Swine_market=1,
                 TAi_pct_Turkeys=1
                 ):
    
    data_dict = data_dict.copy()
    
    
    data_dict['MM_Ti'] = MM_Ti
    data_dict['MM_ASi'] = MM_ASi
    data_dict['MM_LPi'] = MM_LPi
    
    TAi_pct_target = pd.Series(index=['Asses', 'Cattle, dairy','Cattle, non-dairy',
                                  'Chickens, broilers', 'Chickens, layers', 'Ducks', 'Goats',
                                  'Horses', 'Mules', 'Sheep', 'Swine, breeding', 'Swine, market',
                                  'Turkeys'],
                           data=[TAi_Asses_pct, TAi_cattle_dairy_pct,
                                 TAi_pct_Cattle_non_dairy, TAi_Chickens_broilers_pct,
                                 TAi_Chickens_layers_pct, TAi_ducks_pct,TAi_Goat_pct,
                                 TAi_Horses_pct, TAi_Mules_pct ,TAi_Sheep_pct, TAi_Swine_breeding_pct,
                                 TAi_Swine_market_pct, TAi_Turkeys_market_pct
                               
                           ], name='TAi_pct_target')
    
    TAi_pct_target.index.name = 'Item'
    
    tmp = pd.merge(data_dict['TAi'].reset_index(), TAi_pct_target.reset_index(),on='Item')
    tmp['TAi'] = tmp['TAi'] * tmp['TAi_pct_target']

    data_dict['TAi'] = tmp.set_index(['ISO', 'Year', 'Item'])['TAi']
    
    results = model_dictionnary['GE3_model'].run(data_dict)
    
    return results

In [247]:
def run_all_scenarios_GE3(data_dict, ISO, args_dict_1, args_dict_2):
    scenarios_results = {}
    data_dict = {k: v.loc[ISO, 2018, :] for k, v in data_dict.items()}

    scenarios_results['BAU'] = run_scenario(
        data_dict=data_dict, MM_Ti=data_dict['MM_Ti'], MM_ASi=data_dict['MM_ASi'])
    scenarios_results['scenario_one'] = run_scenario(
        data_dict=data_dict, **args_dict_1)
    scenarios_results['scenario_two'] = run_scenario(
        data_dict=data_dict, **args_dict_2)

    df_0 = emission_data_dict_to_df({k: v for k, v in scenarios_results['scenario_one'].items() if k in ['TEE_CO2eq', 'TMA_CO2eq', 'TMT_CO2eq', 'TMP_CO2eq']})
    df_1 = emission_data_dict_to_df({k: v for k, v in scenarios_results['scenario_two'].items() if k in ['TEE_CO2eq', 'TMA_CO2eq', 'TMT_CO2eq', 'TMP_CO2eq']})

    #fig_1 = sankeyplot(df_0, 'Item', 'Variable').update_layout(title='Scenario 1')
    #fig_2 = sankeyplot(df_1, 'Item', 'Variable').update_layout(title='Scenario 2')

    #return fig_1, fig_2, {}
    return df_0, df_1


In [254]:
data_dict = scenario_data_dictionnary['GE3_model']

args_dict_1 = dict(MM_Ti=1/3, MM_ASi=1/3, MM_LPi=1/3)
args_dict_2 = dict(MM_Ti=1/3, MM_ASi=1/3, MM_LPi=1/3, TAi_pct_Cattle_non_dairy=1/2)

df_0, df_1 = run_all_scenarios_GE3(data_dict, 'FRA', args_dict_1, args_dict_2)
df = pd.concat([df_0.assign(scenario='scenario_0'), df_1.assign(scenario='scenario_1')], axis=0)


df = df.merge(model_dictionnary['GE3_model'].summary_df[['name', 'unit']], left_on='Variable', right_index=True)

/home/simon/anaconda3/lib/python3.7/site-packages/pandas/core/indexes/multi.py:3585: RuntimeWarning:

The values in the array are unorderable. Pass `sort=False` to suppress this warning.



In [256]:
df.query('Item == "Cattle, non-dairy"')

,ISO,Year,Item,Value,Variable,scenario,name,unit
4,FRA,2018,"Cattle, non-dairy",1759.820777,TMA_CO2eq,scenario_0,Emissions (CO2eq) (Manure applied),gigagrams (CO2eq)
4,FRA,2018,"Cattle, non-dairy",879.910389,TMA_CO2eq,scenario_1,Emissions (CO2eq) (Manure applied),gigagrams (CO2eq)
23,FRA,2018,"Cattle, non-dairy",2990.464678,TMP_CO2eq,scenario_0,Emissions (CO2eq) (Manure on pasture),gigagrams (CO2eq)
23,FRA,2018,"Cattle, non-dairy",1495.232339,TMP_CO2eq,scenario_1,Emissions (CO2eq) (Manure on pasture),gigagrams (CO2eq)
46,FRA,2018,"Cattle, non-dairy",2973.253021,TMT_CO2eq,scenario_0,Emissions (CO2eq) (Manure management),gigagrams (CO2eq)
46,FRA,2018,"Cattle, non-dairy",1486.626511,TMT_CO2eq,scenario_1,Emissions (CO2eq) (Manure management),gigagrams (CO2eq)
65,FRA,2018,"Cattle, non-dairy",17948.591262,TEE_CO2eq,scenario_0,Emissions (CO2eq) (Enteric),gigagrams (CO2eq)
65,FRA,2018,"Cattle, non-dairy",8974.295631,TEE_CO2eq,scenario_1,Emissions (CO2eq) (Enteric),gigagrams (CO2eq)


In [152]:
def apply_pct_by_item(series, percent_target=1):
    pass

In [176]:
Items = ['Asses', 'Cattle, dairy', 'Cattle, non_dairy',
         'Chickens, broilers', 'Chickens, layers', 'Ducks', 'Goats',
         'Horses', 'Mules', 'Sheep', 'Swine, breeding', 'Swine, market',
         'Turkeys']

sliders_name = [f"TAi_pct_{'_'.join(item.split(', '))}" for item in Items]
sliders_name

['TAi_pct_Asses',
 'TAi_pct_Cattle_dairy',
 'TAi_pct_Cattle_non_dairy',
 'TAi_pct_Chickens_broilers',
 'TAi_pct_Chickens_layers',
 'TAi_pct_Ducks',
 'TAi_pct_Goats',
 'TAi_pct_Horses',
 'TAi_pct_Mules',
 'TAi_pct_Sheep',
 'TAi_pct_Swine_breeding',
 'TAi_pct_Swine_market',
 'TAi_pct_Turkeys']

In [126]:
from ggmodel_dev.models.landuse.GE3 import model_dictionnary


In [136]:
import plotly.express as px
fig = px.treemap(df.query('Variable != "Non-CO2 agricultural emissions"'), path=['Item','name', 'scenario'], values='Value')
fig

In [134]:
fig = px.treemap(df.query('Variable != "Non-CO2 agricultural emissions"'), path=['scenario', 'Item', 'name'], values='Value')
fig

In [131]:
sankeyplot(df, 'Item', 'Variable')

In [360]:
ided_comp =[{'props': {'id': 'MM_Ti_one', 'included': False, 'marks': {'0': {'label': '0%', 'style': {'color': 'white'}}, '1': {'label': '100%', 'style': {'color': 'white'}}}, 'max': 1, 'min': 0, 'step': 0.1, 'value': 0.5, 'drag_value': 0.5}, 'type': 'Slider', 'namespace': 'dash_core_components'}, {'props': {'id': 'MM_ASi_one', 'included': False, 'marks': {'0': {'label': '0%', 'style': {'color': 'white'}}, '1': {'label': '100%', 'style': {'color': 'white'}}}, 'max': 1, 'min': 0, 'step': 0.1, 'value': 1, 'drag_value': 1}, 'type': 'Slider', 'namespace': 'dash_core_components'}, {'props': {'children': [{'props': {'children': [{'props': {'children': [{'props': {'children': 'Asses', 'style': {'font-size': 10}}, 'type': 'P', 'namespace': 'dash_html_components'}, {'props': {'id': 'TAi_pct_Asses_one', 'included': False, 'marks': {'0': {'label': 'x0', 'style': {'color': 'white'}}, '1': {'label': 'x1', 'style': {'color': 'white'}}, '2': {'label': 'x2', 'style': {'color': 'white'}}}, 'max': 2, 'min': 0, 'step': 0.1, 'value': 1, 'drag_value': 1}, 'type': 'Slider', 'namespace': 'dash_core_components'}], 'className': 'row'}, 'type': 'Div', 'namespace': 'dash_html_components'}, {'props': {'children': [{'props': {'children': 'Cattle, dairy', 'style': {'font-size': 10}}, 'type': 'P', 'namespace': 'dash_html_components'}, {'props': {'id': 'TAi_pct_Cattle_dairy_one', 'included': False, 'marks': {}, 'max': 2, 'min': 0, 'step': 0.1, 'value': 1, 'drag_value': 1}, 'type': 'Slider', 'namespace': 'dash_core_components'}], 'className': 'row'}, 'type': 'Div', 'namespace': 'dash_html_components'}, {'props': {'children': [{'props': {'children': 'Cattle, non_dairy', 'style': {'font-size': 10}}, 'type': 'P', 'namespace': 'dash_html_components'}, {'props': {'id': 'TAi_pct_Cattle_non_dairy_one', 'included': False, 'marks': {}, 'max': 2, 'min': 0, 'step': 0.1, 'value': 1, 'drag_value': 1}, 'type': 'Slider', 'namespace': 'dash_core_components'}], 'className': 'row'}, 'type': 'Div', 'namespace': 'dash_html_components'}, {'props': {'children': [{'props': {'children': 'Chickens, broilers', 'style': {'font-size': 10}}, 'type': 'P', 'namespace': 'dash_html_components'}, {'props': {'id': 'TAi_pct_Chickens_broilers_one', 'included': False, 'marks': {}, 'max': 2, 'min': 0, 'step': 0.1, 'value': 1, 'drag_value': 1}, 'type': 'Slider', 'namespace': 'dash_core_components'}], 'className': 'row'}, 'type': 'Div', 'namespace': 'dash_html_components'}], 'className': 'bare_container four columns'}, 'type': 'Div', 'namespace': 'dash_html_components'}, {'props': {'children': [{'props': {'children': [{'props': {'children': 'Chickens, layers', 'style': {'font-size': 10}}, 'type': 'P', 'namespace': 'dash_html_components'}, {'props': {'id': 'TAi_pct_Chickens_layers_one', 'included': False, 'marks': {'0': {'label': 'x0', 'style': {'color': 'white'}}, '1': {'label': 'x1', 'style': {'color': 'white'}}, '2': {'label': 'x2', 'style': {'color': 'white'}}}, 'max': 2, 'min': 0, 'step': 0.1, 'value': 1, 'drag_value': 1}, 'type': 'Slider', 'namespace': 'dash_core_components'}], 'className': 'row'}, 'type': 'Div', 'namespace': 'dash_html_components'}, {'props': {'children': [{'props': {'children': 'Ducks', 'style': {'font-size': 10}}, 'type': 'P', 'namespace': 'dash_html_components'}, {'props': {'id': 'TAi_pct_Ducks_one', 'included': False, 'marks': {}, 'max': 2, 'min': 0, 'step': 0.1, 'value': 1, 'drag_value': 1}, 'type': 'Slider', 'namespace': 'dash_core_components'}], 'className': 'row'}, 'type': 'Div', 'namespace': 'dash_html_components'}, {'props': {'children': [{'props': {'children': 'Goats', 'style': {'font-size': 10}}, 'type': 'P', 'namespace': 'dash_html_components'}, {'props': {'id': 'TAi_pct_Goats_one', 'included': False, 'marks': {}, 'max': 2, 'min': 0, 'step': 0.1, 'value': 1, 'drag_value': 1}, 'type': 'Slider', 'namespace': 'dash_core_components'}], 'className': 'row'}, 'type': 'Div', 'namespace': 'dash_html_components'}, {'props': {'children': [{'props': {'children': 'Horses', 'style': {'font-size': 10}}, 'type': 'P', 'namespace': 'dash_html_components'}, {'props': {'id': 'TAi_pct_Horses_one', 'included': False, 'marks': {}, 'max': 2, 'min': 0, 'step': 0.1, 'value': 1, 'drag_value': 1}, 'type': 'Slider', 'namespace': 'dash_core_components'}], 'className': 'row'}, 'type': 'Div', 'namespace': 'dash_html_components'}], 'className': 'bare_container four columns'}, 'type': 'Div', 'namespace': 'dash_html_components'}, {'props': {'children': [{'props': {'children': [{'props': {'children': 'Sheep', 'style': {'font-size': 10}}, 'type': 'P', 'namespace': 'dash_html_components'}, {'props': {'id': 'TAi_pct_Sheep_one', 'included': False, 'marks': {'0': {'label': 'x0', 'style': {'color': 'white'}}, '1': {'label': 'x1', 'style': {'color': 'white'}}, '2': {'label': 'x2', 'style': {'color': 'white'}}}, 'max': 2, 'min': 0, 'step': 0.1, 'value': 1, 'drag_value': 1}, 'type': 'Slider', 'namespace': 'dash_core_components'}], 'className': 'row'}, 'type': 'Div', 'namespace': 'dash_html_components'}, {'props': {'children': [{'props': {'children': 'Sheep', 'style': {'font-size': 10}}, 'type': 'P', 'namespace': 'dash_html_components'}, {'props': {'id': 'TAi_pct_Sheep_one', 'included': False, 'marks': {}, 'max': 2, 'min': 0, 'step': 0.1, 'value': 1, 'drag_value': 1}, 'type': 'Slider', 'namespace': 'dash_core_components'}], 'className': 'row'}, 'type': 'Div', 'namespace': 'dash_html_components'}, {'props': {'children': [{'props': {'children': 'Swine, breeding', 'style': {'font-size': 10}}, 'type': 'P', 'namespace': 'dash_html_components'}, {'props': {'id': 'TAi_pct_Swine_breeding_one', 'included': False, 'marks': {}, 'max': 2, 'min': 0, 'step': 0.1, 'value': 1, 'drag_value': 1}, 'type': 'Slider', 'namespace': 'dash_core_components'}], 'className': 'row'}, 'type': 'Div', 'namespace': 'dash_html_components'}, {'props': {'children': [{'props': {'children': 'Swine, market', 'style': {'font-size': 10}}, 'type': 'P', 'namespace': 'dash_html_components'}, {'props': {'id': 'TAi_pct_Swine_market_one', 'included': False, 'marks': {}, 'max': 2, 'min': 0, 'step': 0.1, 'value': 1, 'drag_value': 1}, 'type': 'Slider', 'namespace': 'dash_core_components'}], 'className': 'row'}, 'type': 'Div', 'namespace': 'dash_html_components'}], 'className': 'bare_container four columns'}, 'type': 'Div', 'namespace': 'dash_html_components'}], 'id': 'TAi_pct_one'}, 'type': 'Div', 'namespace': 'dash_html_components'}]

In [367]:
def get_ided_comp_from_box(box):
    return [el for el in box['props']['children'] if 'id' in el['props']]

In [376]:
get_ided_comp_from_box(ided_comp[2]['props']['children'][0])

[]

In [361]:
ided_comp 

[{'props': {'id': 'MM_Ti_one',
   'included': False,
   'marks': {'0': {'label': '0%', 'style': {'color': 'white'}},
    '1': {'label': '100%', 'style': {'color': 'white'}}},
   'max': 1,
   'min': 0,
   'step': 0.1,
   'value': 0.5,
   'drag_value': 0.5},
  'type': 'Slider',
  'namespace': 'dash_core_components'},
 {'props': {'id': 'MM_ASi_one',
   'included': False,
   'marks': {'0': {'label': '0%', 'style': {'color': 'white'}},
    '1': {'label': '100%', 'style': {'color': 'white'}}},
   'max': 1,
   'min': 0,
   'step': 0.1,
   'value': 1,
   'drag_value': 1},
  'type': 'Slider',
  'namespace': 'dash_core_components'},
 {'props': {'children': [{'props': {'children': [{'props': {'children': [{'props': {'children': 'Asses',
            'style': {'font-size': 10}},
           'type': 'P',
           'namespace': 'dash_html_components'},
          {'props': {'id': 'TAi_pct_Asses_one',
            'included': False,
            'marks': {'0': {'label': 'x0', 'style': {'color': 'white'}}